In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
data_binary = pd.read_csv(r'.\TrafficLabelling\data_binary.csv')
data_multiclass = pd.read_csv(r'.\TrafficLabelling\data_multi_class.csv')

In [53]:
data_binary.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.00,0.00,0,0,0.00,0.00,"4,000,000.00","666,666.67",3.00,0.00,3,3,3,3.00,0.00,3,3,0,0.00,0.00,0,0,0,0,0,0,40,0,"666,666.67",0.00,6,6,6.00,0.00,0.00,0,0,0,0,1,0,0,0,0,9.00,6.00,0.00,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.00,0.00,0,0,0.00,0.00,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.00,0.00,6,6,6.00,0.00,"110,091.74","18,348.62",109.00,0.00,109,109,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0,0,0,20,20,"9,174.31","9,174.31",6,6,6.00,0.00,0.00,0,0,0,0,1,1,0,0,1,9.00,6.00,6.00,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.00,0.00,0,0,0.00,0.00,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.00,0.00,6,6,6.00,0.00,"230,769.23","38,461.54",52.00,0.00,52,52,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0,0,0,20,20,"19,230.77","19,230.77",6,6,6.00,0.00,0.00,0,0,0,0,1,1,0,0,1,9.00,6.00,6.00,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.00,0.00,0,0,0.00,0.00,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.00,0.00,6,6,6.00,0.00,"352,941.18","58,823.53",34.00,0.00,34,34,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0,0,0,20,20,"29,411.76","29,411.76",6,6,6.00,0.00,0.00,0,0,0,0,1,1,0,0,1,9.00,6.00,6.00,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.00,0.00,0,0,0.00,0.00,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.00,0.00,0,0,0.00,0.00,"4,000,000.00","666,666.67",3.00,0.00,3,3,3,3.00,0.00,3,3,0,0.00,0.00,0,0,0,0,0,0,40,0,"666,666.67",0.00,6,6,6.00,0.00,0.00,0,0,0,0,1,0,0,0,0,9.00,6.00,0.00,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.00,0.00,0,0,0.00,0.00,0,0,BENIGN


In [54]:
pd.set_option('display.max_rows', None)  # hiển thị tất cả dòng
pd.set_option('display.max_columns', None)  # hiển thị tất cả cột
pd.set_option("display.float_format", "{:.2f}".format)

In [55]:
summary = data_binary.describe().T[['min','max','mean']]
# Hiển thị toàn bộ bảng (không bị cắt)
pd.set_option('display.max_rows', None)  # hiển thị tất cả dòng
pd.set_option('display.max_columns', None)  # hiển thị tất cả cột

# Hiển thị số ở dạng thường, làm tròn 2 chữ số thập phân
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print(summary)


                                           min              max          mean
Destination Port                          0.00        65,535.00      8,077.64
Flow Duration                           -13.00   119,999,998.00 14,792,023.78
Total Fwd Packets                         1.00       219,759.00          9.36
Total Backward Packets                    0.00       291,922.00         10.40
Total Length of Fwd Packets               0.00    12,900,000.00        548.32
Total Length of Bwd Packets               0.00   655,453,030.00     16,172.19
Fwd Packet Length Max                     0.00        24,820.00        207.72
Fwd Packet Length Min                     0.00         2,325.00         18.73
Fwd Packet Length Mean                    0.00         5,940.86         58.24
Fwd Packet Length Std                     0.00         7,125.60         68.95
Bwd Packet Length Max                     0.00        19,530.00        871.44
Bwd Packet Length Min                     0.00         2,896.00 

In [56]:
print("Tổng số giá trị thiếu:", data_binary.isnull().sum().sum())
print("Tổng số giá trị thiếu:", data_multiclass.isnull().sum().sum())

Tổng số giá trị thiếu: 0
Tổng số giá trị thiếu: 0


In [57]:
all_zero_cols = [col for col in data_binary.columns if (data_binary[col].nunique() == 1 and data_binary[col].iloc[0] == 0)]

print("Các cột toàn 0:")
for col in all_zero_cols:
    print(f"- {col}")


Các cột toàn 0:
- Bwd PSH Flags
- Bwd URG Flags
- Fwd Avg Bytes/Bulk
- Fwd Avg Packets/Bulk
- Fwd Avg Bulk Rate
- Bwd Avg Bytes/Bulk
- Bwd Avg Packets/Bulk
- Bwd Avg Bulk Rate


In [58]:
data_binary = data_binary.drop(columns=all_zero_cols)
data_multiclass = data_multiclass.drop(columns=all_zero_cols)

In [59]:
numeric_cols = data_binary.select_dtypes(include="number")
neg_count = (numeric_cols < 0).sum()
neg_percent = neg_count / len(data_binary) * 100
result = pd.DataFrame({
    "Số âm": neg_count,
    "Tỷ lệ %": neg_percent
})
result["Tỷ lệ %"] = result["Tỷ lệ %"].map(lambda x: f"{x:.10f}")
print(result)


                               Số âm        Tỷ lệ %
Destination Port                   0   0.0000000000
Flow Duration                    115   0.0040656687
Total Fwd Packets                  0   0.0000000000
Total Backward Packets             0   0.0000000000
Total Length of Fwd Packets        0   0.0000000000
Total Length of Bwd Packets        0   0.0000000000
Fwd Packet Length Max              0   0.0000000000
Fwd Packet Length Min              0   0.0000000000
Fwd Packet Length Mean             0   0.0000000000
Fwd Packet Length Std              0   0.0000000000
Bwd Packet Length Max              0   0.0000000000
Bwd Packet Length Min              0   0.0000000000
Bwd Packet Length Mean             0   0.0000000000
Bwd Packet Length Std              0   0.0000000000
Flow Bytes/s                      85   0.0030050595
Flow Packets/s                   115   0.0040656687
Flow IAT Mean                    115   0.0040656687
Flow IAT Std                       0   0.0000000000
Flow IAT Max

In [60]:
data_binary = data_binary.drop(columns=["Init_Win_bytes_backward", "Init_Win_bytes_forward"])
data_multiclass = data_multiclass.drop(columns=["Init_Win_bytes_backward", "Init_Win_bytes_forward"])

In [61]:
# Chọn các cột số
numeric_cols = data_binary.select_dtypes(include="number")

# Giữ lại chỉ những hàng mà tất cả giá trị số đều >= 0
data_binary = data_binary[(numeric_cols >= 0).all(axis=1)]
data_multiclass = data_multiclass[(numeric_cols >= 0).all(axis=1)]


In [62]:
print(data_binary["Label"].value_counts())
print(data_multiclass["Label"].value_counts())

Label
BENIGN    2270365
ATTACK     555272
Name: count, dtype: int64
Label
BENIGN            2270365
DOS_DDOS           380505
PROBE_PORTSCAN     158930
CRED_WEB            13829
R2L_U2R_OTHER        2008
Name: count, dtype: int64


In [63]:
data_binary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2825637 entries, 0 to 2828562
Data columns (total 69 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             int64  
 1   Flow Duration                int64  
 2   Total Fwd Packets            int64  
 3   Total Backward Packets       int64  
 4   Total Length of Fwd Packets  int64  
 5   Total Length of Bwd Packets  int64  
 6   Fwd Packet Length Max        int64  
 7   Fwd Packet Length Min        int64  
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        int64  
 11  Bwd Packet Length Min        int64  
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 int64  
 19  Flow 

In [64]:
data_binary.describe().T

,count,mean,std,min,25%,50%,75%,max
Destination Port,"2,825,637.00","8,082.87","18,294.53",0.00,53.00,80.00,443.00,"65,535.00"
Flow Duration,"2,825,637.00","14,748,704.34","33,618,106.59",0.00,155.00,"31,286.00","3,073,944.00","119,999,998.00"
Total Fwd Packets,"2,825,637.00",5.73,74.35,1.00,2.00,2.00,5.00,"112,141.00"
Total Backward Packets,"2,825,637.00",5.55,102.96,0.00,1.00,2.00,4.00,"157,388.00"
Total Length of Fwd Packets,"2,825,637.00",526.25,"9,116.50",0.00,12.00,62.00,187.00,"12,900,000.00"
Total Length of Bwd Packets,"2,825,637.00","5,343.43","250,509.29",0.00,2.00,122.00,480.00,"399,000,000.00"
Fwd Packet Length Max,"2,825,637.00",207.18,716.59,0.00,6.00,37.00,80.00,"24,820.00"
Fwd Packet Length Min,"2,825,637.00",18.75,60.39,0.00,0.00,2.00,36.00,"2,325.00"
Fwd Packet Length Mean,"2,825,637.00",58.21,186.05,0.00,6.00,34.00,50.00,"5,940.86"
Fwd Packet Length Std,"2,825,637.00",68.85,281.16,0.00,0.00,0.00,26.16,"7,125.60"


In [65]:
numeric_cols = data_binary.select_dtypes(include="number")

corr_matrix = numeric_cols.corr().abs()

# Lấy phần tam giác trên (để tránh trùng lặp và loại đường chéo = 1)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Tìm các cặp có corr > 0.95
high_corr = [
    (col, row, upper.loc[row, col])
    for col in upper.columns
    for row in upper.index
    if pd.notna(upper.loc[row, col]) and upper.loc[row, col] > 0.90
]

# Chuyển thành DataFrame để xem dễ hơn
high_corr_df = pd.DataFrame(high_corr, columns=["Feature 1", "Feature 2", "Correlation"])

print(high_corr_df.sort_values(by="Correlation", ascending=False))

                      Feature 1                    Feature 2  Correlation
35               CWE Flag Count                Fwd URG Flags         1.00
34               SYN Flag Count                Fwd PSH Flags         1.00
62          Subflow Bwd Packets       Total Backward Packets         1.00
41         Avg Fwd Segment Size       Fwd Packet Length Mean         1.00
52          Fwd Header Length.1            Fwd Header Length         1.00
54          Subflow Fwd Packets            Total Fwd Packets         1.00
43         Avg Bwd Segment Size       Bwd Packet Length Mean         1.00
70            Subflow Bwd Bytes  Total Length of Bwd Packets         1.00
60            Subflow Fwd Bytes  Total Length of Fwd Packets         1.00
8                 Fwd IAT Total                Flow Duration         1.00
12                  Fwd IAT Max                 Flow IAT Max         1.00
39          Average Packet Size           Packet Length Mean         1.00
36               ECE Flag Count       

In [66]:
drop_collums=["CWE Flag Count","SYN Flag Count","Subflow Bwd Packets","Avg Fwd Segment Size","Fwd Header Length.1",
              "Subflow Fwd Packets","Avg Bwd Segment Size","Subflow Bwd Bytes","Subflow Fwd Bytes","Fwd IAT Total",
              "Fwd IAT Max","Average Packet Size","ECE Flag Count","Bwd Header Length","Idle Min","Idle Mean","Total Backward Packets",
              "Fwd Packets/s","Fwd Header Length","Fwd Packet Length Std","act_data_pkt_fwd","Total Length of Bwd Packets",
              "Idle Max","Packet Length Std","Bwd Packet Length Std","Bwd Packet Length Mean","Packet Length Variance","Active Min",
              "Bwd Packet Length Max","Flow IAT Max","Bwd IAT Min","Max Packet Length","Fwd IAT Mean"]
data_binary = data_binary.drop(columns=drop_collums)
data_multiclass = data_multiclass.drop(columns=drop_collums)

In [67]:
data_binary.describe().T

,count,mean,std,min,25%,50%,75%,max
Destination Port,"2,825,637.00","8,082.87","18,294.53",0.00,53.00,80.00,443.00,"65,535.00"
Flow Duration,"2,825,637.00","14,748,704.34","33,618,106.59",0.00,155.00,"31,286.00","3,073,944.00","119,999,998.00"
Total Fwd Packets,"2,825,637.00",5.73,74.35,1.00,2.00,2.00,5.00,"112,141.00"
Total Length of Fwd Packets,"2,825,637.00",526.25,"9,116.50",0.00,12.00,62.00,187.00,"12,900,000.00"
Fwd Packet Length Max,"2,825,637.00",207.18,716.59,0.00,6.00,37.00,80.00,"24,820.00"
Fwd Packet Length Min,"2,825,637.00",18.75,60.39,0.00,0.00,2.00,36.00,"2,325.00"
Fwd Packet Length Mean,"2,825,637.00",58.21,186.05,0.00,6.00,34.00,50.00,"5,940.86"
Bwd Packet Length Min,"2,825,637.00",41.12,68.90,0.00,0.00,0.00,77.00,"2,896.00"
Flow Bytes/s,"2,825,637.00","1,492,814.16","25,949,335.15",0.00,119.22,"4,581.75","166,666.67","2,071,000,000.00"
Flow Packets/s,"2,825,637.00","70,954.79","254,121.44",0.00,3.40,109.75,"23,255.81","4,000,000.00"


### Scale

In [68]:
data_binary.shape

(2825637, 36)

In [69]:
# 1. Nhóm feature dạng count/flag (0/1 hoặc nhỏ)
flag_features = ['Fwd PSH Flags', 'Fwd URG Flags', 'FIN Flag Count',
                 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count']

# 2. Nhóm feature có outlier lớn, lệch mạnh
outlier_features = ['Flow Duration', 'Flow Bytes/s', 'Flow Packets/s',
                    'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Min',
                    'Fwd IAT Std', 'Fwd IAT Min',
                    'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                    'Active Mean', 'Active Std', 'Active Max', 'Idle Std']

# 3. Nhóm còn lại (Port, Length, Ratio, Packet stats…) outlier vừa phải, lệnh vừa
other_features = [col for col in data_binary.columns if col not in flag_features + outlier_features +["Label"] ]
other_features

['Destination Port',
 'Total Fwd Packets',
 'Total Length of Fwd Packets',
 'Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packets/s',
 'Min Packet Length',
 'Packet Length Mean',
 'Down/Up Ratio',
 'min_seg_size_forward']

In [70]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [71]:
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Đảm bảo đầu vào là DataFrame hoặc ndarray
        X = np.asarray(X, dtype=np.float64)
        # Clip để tránh log âm và log(0)
        X = np.clip(X, a_min=0, a_max=None)
        return np.log1p(X)
flag_pipeline = 'passthrough'

# Nhóm 2: outlier -> log rồi StandardScaler
outlier_pipeline = Pipeline([
    ('log', LogTransformer()),
    ('scale', StandardScaler())
])

# Nhóm 3: other -> RobustScaler để giảm outlier
other_pipeline = Pipeline([
    ('scale', RobustScaler())
])

# ===== 4. Kết hợp lại thành ColumnTransformer =====
preprocessor = ColumnTransformer(
    transformers=[
        ('flag', flag_pipeline, flag_features),
        ('outlier', outlier_pipeline, outlier_features),
        ('other', other_pipeline, other_features)
    ],
    remainder='drop'  # bỏ các cột khác (Label)
)

# ===== 5. Tạo pipeline tổng =====
scaling_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor)
])



In [72]:
X_binary = data_binary.drop(columns=['Label'])
y_binary = data_binary['Label']
X_multiclass = data_multiclass.drop(columns=['Label'])
y_multiclass = data_multiclass['Label']

X_binary_scaled = scaling_pipeline.fit_transform(X_binary)
X_multiclass_scaled = scaling_pipeline.transform(X_multiclass)

# trả về DataFrame:
scaled_df_binary = pd.DataFrame(X_binary_scaled, columns=flag_features + outlier_features + other_features)
scaled_df_binary['Label'] = y_binary.values
scaled_df_multiclass = pd.DataFrame(X_multiclass_scaled, columns=flag_features + outlier_features + other_features)
scaled_df_multiclass['Label'] = y_multiclass.values
scaled_df_binary.head()

,Fwd PSH Flags,Fwd URG Flags,FIN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Min,Fwd IAT Std,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Active Mean,Active Std,Active Max,Idle Std,Destination Port,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Min,Bwd Packets/s,Min Packet Length,Packet Length Mean,Down/Up Ratio,min_seg_size_forward,Label
0,0.00,0.00,0.00,0.00,0.00,1.00,0.00,-1.56,1.51,1.73,-1.57,-1.09,-0.60,-0.72,-0.36,-0.75,-0.76,-0.61,-0.76,-0.47,-0.27,-0.47,-0.29,140.47,0.00,-0.29,-0.42,0.11,-0.64,0.00,-0.00,0.11,-0.45,-1.00,-0.33,BENIGN
1,0.00,0.00,0.00,0.00,0.00,1.00,1.00,-0.96,0.76,0.91,-0.86,-1.09,0.36,-0.72,-0.75,-0.75,-0.76,-0.61,-0.76,-0.47,-0.27,-0.47,-0.29,140.96,-0.33,-0.32,-0.42,0.11,-0.64,0.08,1.24,0.11,-0.45,0.00,-0.33,BENIGN
2,0.00,0.00,0.00,0.00,0.00,1.00,1.00,-1.09,0.91,1.08,-1.02,-1.09,0.15,-0.72,-0.75,-0.75,-0.76,-0.61,-0.76,-0.47,-0.27,-0.47,-0.29,140.96,-0.33,-0.32,-0.42,0.11,-0.64,0.08,2.60,0.11,-0.45,0.00,-0.33,BENIGN
3,0.00,0.00,0.00,0.00,0.00,1.00,1.00,-1.16,1.00,1.17,-1.11,-1.09,0.03,-0.72,-0.75,-0.75,-0.76,-0.61,-0.76,-0.47,-0.27,-0.47,-0.29,118.35,-0.33,-0.32,-0.42,0.11,-0.64,0.08,3.98,0.11,-0.45,0.00,-0.33,BENIGN
4,0.00,0.00,0.00,0.00,0.00,1.00,0.00,-1.56,1.51,1.73,-1.57,-1.09,-0.60,-0.72,-0.36,-0.75,-0.76,-0.61,-0.76,-0.47,-0.27,-0.47,-0.29,140.47,0.00,-0.29,-0.42,0.11,-0.64,0.00,-0.00,0.11,-0.45,-1.00,-0.33,BENIGN


In [73]:
scaled_df_binary.describe().T

,count,mean,std,min,25%,50%,75%,max
Fwd PSH Flags,"2,825,637.00",0.05,0.21,0.00,0.00,0.00,0.00,1.00
Fwd URG Flags,"2,825,637.00",0.00,0.01,0.00,0.00,0.00,0.00,1.00
FIN Flag Count,"2,825,637.00",0.04,0.18,0.00,0.00,0.00,0.00,1.00
RST Flag Count,"2,825,637.00",0.00,0.02,0.00,0.00,0.00,0.00,1.00
PSH Flag Count,"2,825,637.00",0.30,0.46,0.00,0.00,0.00,1.00,1.00
ACK Flag Count,"2,825,637.00",0.32,0.46,0.00,0.00,0.00,1.00,1.00
URG Flag Count,"2,825,637.00",0.09,0.29,0.00,0.00,0.00,0.00,1.00
Flow Duration,"2,825,637.00",-0.00,1.00,-1.82,-0.89,0.08,0.92,1.59
Flow Bytes/s,"2,825,637.00",0.00,1.00,-1.67,-0.67,0.09,0.84,2.81
Flow Packets/s,"2,825,637.00",0.00,1.00,-1.33,-0.99,-0.26,0.96,2.13


In [74]:
scaled_df_binary.to_csv(r'.\TrafficLabelling\data_binary_scaled.csv', index=False)
scaled_df_binary.to_csv(r'.\TrafficLabelling\data_multi_class_scaled.csv', index=False)

In [75]:
joblib.dump(scaling_pipeline, "scaling_pipeline.pkl")

['scaling_pipeline.pkl']